# Feature Importance Using Random Forest Classification & PySpark

Predict which chemical preservative is affecting dog food batches the most using Random Forest

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled

In [0]:
# Start a spark session
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('dog').getOrCreate()

In [0]:
# Import the datafile
data=spark.read.csv('dbfs:/FileStore/shared_uploads/hrishagni95@gmail.com/dog_food-1.csv',inferSchema=True,header=True)

In [0]:
# Sneak peek of the dataset
data.show()

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
 10| 3|13.0| 9| 1.0|
 8| 5|14.0| 5| 1.0|
 5| 8|12.0| 8| 1.0|
 6| 5|12.0| 9| 1.0|
 3| 3|12.0| 1| 1.0|
 9| 8|11.0| 3| 1.0|
 1| 10|12.0| 3| 1.0|
 1| 5|13.0| 10| 1.0|
 2| 10|12.0| 6| 1.0|
 1| 10|11.0| 4| 1.0|
 5| 3|12.0| 2| 1.0|
 4| 9|11.0| 8| 1.0|
 5| 1|11.0| 1| 1.0|
 4| 9|12.0| 10| 1.0|
 5| 8|10.0| 9| 1.0|
+---+---+----+---+-------+
only showing top 20 rows

In [0]:
# Import RandomForestClassifier from pyspark
from pyspark.ml.classification import RandomForestClassifier

In [0]:
# Import VectorAssembler from pyspark
from pyspark.ml.feature import VectorAssembler

In [0]:
# Create an instance of VectorAssembler
assembler=VectorAssembler(inputCols=['A','B','C','D'],outputCol='features')

In [0]:
# Transform the data to create a features column to be used for classification
output=assembler.transform(data)

In [0]:
# Create an instance of RandomForestClassifier
rfc=RandomForestClassifier(labelCol='Spoiled',featuresCol='features')

In [0]:
output.printSchema()

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
# Filter the transformed data set
final_data=output.select('features','Spoiled')

In [0]:
final_data.show()

+-------------------+-------+
 features|Spoiled|
+-------------------+-------+
 [4.0,2.0,12.0,3.0]| 1.0|
 [5.0,6.0,12.0,7.0]| 1.0|
 [6.0,2.0,13.0,6.0]| 1.0|
 [4.0,2.0,12.0,1.0]| 1.0|
 [4.0,2.0,12.0,3.0]| 1.0|
[10.0,3.0,13.0,9.0]| 1.0|
 [8.0,5.0,14.0,5.0]| 1.0|
 [5.0,8.0,12.0,8.0]| 1.0|
 [6.0,5.0,12.0,9.0]| 1.0|
 [3.0,3.0,12.0,1.0]| 1.0|
 [9.0,8.0,11.0,3.0]| 1.0|
[1.0,10.0,12.0,3.0]| 1.0|
[1.0,5.0,13.0,10.0]| 1.0|
[2.0,10.0,12.0,6.0]| 1.0|
[1.0,10.0,11.0,4.0]| 1.0|
 [5.0,3.0,12.0,2.0]| 1.0|
 [4.0,9.0,11.0,8.0]| 1.0|
 [5.0,1.0,11.0,1.0]| 1.0|
[4.0,9.0,12.0,10.0]| 1.0|
 [5.0,8.0,10.0,9.0]| 1.0|
+-------------------+-------+
only showing top 20 rows

In [0]:
# Fit the filtered data on the RandomForestClassifier model
rfc_model=rfc.fit(final_data)

In [0]:
# Display the feature importances
rfc_model.featureImportances

Out[13]: SparseVector(4, {0: 0.0189, 1: 0.0141, 2: 0.9384, 3: 0.0287})

In [0]:
# As we can see feature 2 or chemical C is the most important in affecting the dog food batches